# Lambda Code to stop a notebook instance

In [1]:
import boto3
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


Let's check the SageMaker API and see how can we find the instances that are running in the account

In [17]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()
candidates

{'NotebookInstances': [{'NotebookInstanceName': 'fastai-with-efs',
   'NotebookInstanceArn': 'arn:aws:sagemaker:eu-west-1:217431963147:notebook-instance/fastai-with-efs',
   'NotebookInstanceStatus': 'InService',
   'Url': 'fastai-with-efs.notebook.eu-west-1.sagemaker.aws',
   'InstanceType': 'ml.m4.2xlarge',
   'CreationTime': datetime.datetime(2018, 8, 13, 13, 13, 13, 863000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2018, 8, 23, 7, 54, 9, 838000, tzinfo=tzlocal()),
   'NotebookInstanceLifecycleConfigName': 'MountEFS'}],
 'ResponseMetadata': {'RequestId': 'ab00e2a4-25b3-4d32-b770-fc4cea6d7111',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ab00e2a4-25b3-4d32-b770-fc4cea6d7111',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '407',
   'date': 'Thu, 23 Aug 2018 11:41:47 GMT'},
  'RetryAttempts': 0}}

We want to find the name of the instance. It is used to send the stop and start commands

In [24]:
for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']

candidateName

'fastai-with-efs'

To discover the tags of the notebook instances we need to know the ARN of each

In [30]:
instanceArn = candidate['NotebookInstanceArn']
instanceTags = client.list_tags(ResourceArn=instanceArn)
instanceTags['Tags']

[{'Key': 'Demo', 'Value': 'fastai'}]

We will look for a tag with key of 'InDuty' and a value of 'Yes'

In [33]:
for tag in instanceTags['Tags']:
    if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
        print("Found it")

Now we can connect it all together

In [35]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()

for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']
    logger.info('instance {0} is in Service'.format(candidateName))
    instanceArn = candidate['NotebookInstanceArn']
    instanceTags = client.list_tags(ResourceArn=instanceArn)
    for tag in instanceTags['Tags']:
        if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
            logger.info('instance {0} is in duty'.format(candidateName))
    #        response = client.stop_notebook_instance(NotebookInstanceName=candidateName)
            logger.info('instance {0} was stopped'.format(candidateName))
        else:
            logger.info('Not in duty'.format(candidateName))


# Creating a lambda function

# Creating IAM Role